In [1]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms, utils
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt
import scipy.misc
from PIL import Image
import json


In [34]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
model.name = 'resnet18'

Using cache found in C:\Users\dingb/.cache\torch\hub\pytorch_vision_v0.10.0


In [35]:
print(model.name)

resnet18


In [4]:
# we will save the conv layer weights in this list
model_weights =[]
#we will save the 49 conv layers in this list
conv_layers = []
# get all the model children as list
model_children = list(model.children())
#counter to keep count of the conv layers
counter = 0
#append all the conv layers and their respective wights to the list
for i in range(len(model_children)):
    if type(model_children[i]) == nn.Conv2d:
        counter+=1
        model_weights.append(model_children[i].weight)
        conv_layers.append(model_children[i])
    elif type(model_children[i]) == nn.Sequential:
        for j in range(len(model_children[i])):
            for child in model_children[i][j].children():
                if type(child) == nn.Conv2d:
                    counter+=1
                    model_weights.append(child.weight)
                    conv_layers.append(child)
print(f"Total convolution layers: {counter}")
print("conv_layers")

Total convolution layers: 17
conv_layers


In [25]:
from torchvision.ops import roi_align
import math
import torch.nn as nn
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models.feature_extraction import get_graph_node_names

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [26]:
train_nodes, eval_nodes = get_graph_node_names(model)

In [27]:
print(train_nodes)

['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.add', 'layer2.1.relu_1', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.0.add', 'layer3.0.relu_1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer3.1.add', 'layer3.1.relu_1', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'lay

In [28]:
print(eval_nodes)

['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.add', 'layer1.0.relu_1', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.add', 'layer1.1.relu_1', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_1', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.add', 'layer2.1.relu_1', 'layer3.0.conv1', 'layer3.0.bn1', 'layer3.0.relu', 'layer3.0.conv2', 'layer3.0.bn2', 'layer3.0.downsample.0', 'layer3.0.downsample.1', 'layer3.0.add', 'layer3.0.relu_1', 'layer3.1.conv1', 'layer3.1.bn1', 'layer3.1.relu', 'layer3.1.conv2', 'layer3.1.bn2', 'layer3.1.add', 'layer3.1.relu_1', 'layer4.0.conv1', 'layer4.0.bn1', 'layer4.0.relu', 'layer4.0.conv2', 'layer4.0.bn2', 'layer4.0.downsample.0', 'layer4.0.downsample.1', 'lay

In [29]:
return_nodes = {
    # node_name: user-specified key for output dict
    'layer1.1.relu_1': 'layer1',
    'layer2.1.relu_1': 'layer2',
    'layer3.1.relu_1': 'layer3',
    'layer4.1.relu_1': 'layer4',
}

In [30]:
create_feature_extractor(model, return_nodes=return_nodes)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Module(
    (0): Module(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Module(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2):

In [7]:
preprocess = transforms.Compose([
    # transforms.Resize((1280,720)),
    # transforms.CenterCrop(1280),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = Image.open('dst/fisheye/test/0.jpg')
image = preprocess(image)
img_shape = image.shape
print(img_shape)
print(f"Image shape before: {image.shape}")
image = image.unsqueeze(0)
print(f"Image shape after: {image.shape}")
image = image.to(device)

torch.Size([3, 960, 1280])
Image shape before: torch.Size([3, 960, 1280])
Image shape after: torch.Size([1, 3, 960, 1280])


In [8]:
outputs = []
names = []
for layer in conv_layers[0:]:
    image = layer(image)
    outputs.append(image)
    names.append(str(layer))
print(len(outputs))
#print feature_maps
for feature_map in outputs:
    print(feature_map.shape)
feature = outputs[-1]

17
torch.Size([1, 64, 480, 640])
torch.Size([1, 64, 480, 640])
torch.Size([1, 64, 480, 640])
torch.Size([1, 64, 480, 640])
torch.Size([1, 64, 480, 640])
torch.Size([1, 128, 240, 320])
torch.Size([1, 128, 240, 320])
torch.Size([1, 128, 240, 320])
torch.Size([1, 128, 240, 320])
torch.Size([1, 256, 120, 160])
torch.Size([1, 256, 120, 160])
torch.Size([1, 256, 120, 160])
torch.Size([1, 256, 120, 160])
torch.Size([1, 512, 60, 80])
torch.Size([1, 512, 60, 80])
torch.Size([1, 512, 60, 80])
torch.Size([1, 512, 60, 80])


In [9]:
print(feature.size())

torch.Size([1, 512, 60, 80])


In [10]:
box = [torch.tensor([[ 382., 790., 437., 866.]]).to(device)]
w, l = 437-382, 866-790
scale = feature.shape[2]/img_shape[1]
l = l*scale
w = w*scale
output_size = (int(math.ceil(w)), int(math.ceil(l)))
print(output_size)
aligned_f = roi_align(feature, box, output_size= output_size, spatial_scale=scale, aligned=True)
print(aligned_f.shape)

(4, 5)
torch.Size([1, 512, 4, 5])


In [11]:
from torch.nn import AdaptiveMaxPool2d

In [12]:
m = AdaptiveMaxPool2d((2, 2))
out = m(aligned_f)

In [13]:
out.size()

torch.Size([1, 512, 2, 2])

In [14]:
final_f = torch.flatten(out)


In [15]:
relu_layer = nn.ReLU()
o = relu_layer(final_f)

In [19]:
# num class 2 for road object and not road object
cls_score = nn.Linear(2048, 2).to(device)
bbox_pred = nn.Linear(2048, 2 * 4).to(device)

In [20]:
cls_score = cls_score(final_f)
bbox_pred = bbox_pred(final_f)
print(cls_score)

tensor([17058.2988,  7212.7549], device='cuda:0', grad_fn=<AddBackward0>)


In [21]:
print(bbox_pred)

tensor([-22822.5723,  13268.3457,  33149.7109, -19445.3262,  46201.9258,
         28258.6973, -33579.6836, -20196.9043], device='cuda:0',
       grad_fn=<AddBackward0>)


In [ ]:
class FasterRCNNBase(nn.Module):
    """
    Main class for Generalized R-CNN.

    Arguments:
        backbone (nn.Module):
        rpn (nn.Module):
        roi_heads (nn.Module): takes the features + the proposals from the RPN and computes
            detections / masks from it.
        transform (nn.Module): performs the data transformation from the inputs to feed into
            the model
    """

    def __init__(self, backbone, rpn, roi_heads, transform):
        super(FasterRCNNBase, self).__init__()
        self.transform = transform
        self.backbone = backbone
        self.rpn = rpn
        self.roi_heads = roi_heads

    @torch.jit.unused
    def eager_outputs(self, losses, detections):
        if self.training:
            return losses

        return detections

    def forward(self, images, targets=None):
        """
        Arguments:
            images (list[Tensor]): images to be processed
            targets (list[Dict[Tensor]]): ground-truth boxes present in the image (optional)

        Returns:
            result (list[BoxList] or dict[Tensor]): the output from the model.
                During training, it returns a dict[Tensor] which contains the losses.
                During testing, it returns list[BoxList] contains additional fields
                like `scores`, `labels` and `mask` (for Mask R-CNN models).

        """
        if self.training and targets is None:
            raise ValueError("In training mode, targets should be passed")

        if self.training:
            assert targets is not None
            for target in targets:
                boxes = target["boxes"]
                if isinstance(boxes, torch.Tensor):
                    if len(boxes.shape) != 2 or boxes.shape[-1] != 4:
                        raise ValueError("Expected target boxes to be a tensor"
                                         "of shape [N, 4], got {:}.".format(
                            boxes.shape))
                else:
                    raise ValueError("Expected target boxes to be of type "
                                     "Tensor, got {:}.".format(type(boxes)))

        original_image_sizes = torch.jit.annotate(List[Tuple[int, int]], [])
        for img in images:
            val = img.shape[-2:]
            assert len(val) == 2
            original_image_sizes.append((val[0], val[1]))

        images, targets = self.transform(images, targets)

        features = self.backbone(images.tensors)
        if isinstance(features, torch.Tensor):
            features = OrderedDict([('0', features)])

        proposals, proposal_losses = self.rpn(images, features, targets)

        detections, detector_losses = self.roi_heads(features, proposals, images.image_sizes, targets)

        detections = self.transform.postprocess(detections, images.image_sizes, original_image_sizes)

        losses = {}
        losses.update(detector_losses)
        losses.update(proposal_losses)

        return self.eager_outputs(losses, detections)

In [45]:
class FastRCNN_GMM(nn.Module):
    def __init__(self, backbone, num_class):
        super(FastRCNN_GMM, self).__init__()
        if backbone.name == 'resnet18':
            return_nodes = {
                # node_name: user-specified key for output dict
                'layer1.1.relu_1': 'layer1',
                'layer2.1.relu_1': 'layer2',
                'layer3.1.relu_1': 'layer3',
                'layer4.1.relu_1': 'layer4',
                }
            self.backbone = create_feature_extractor(model, return_nodes=return_nodes)
        cls_score = nn.Linear(2048, 2)
        bbox_pred = nn.Linear(2048, 2 * 4)
    def forward(self, x):
        x = self.backbone(x)
        return x
        

In [51]:
m = FastRCNN_GMM(model, 2).to(device)

In [52]:
preprocess = transforms.Compose([
    # transforms.Resize((1280,720)),
    # transforms.CenterCrop(1280),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

image = Image.open('dst/fisheye/test/0.jpg')
image = preprocess(image)
image = image.unsqueeze(0)
image = image.to(device)

In [57]:
o = m(image)

In [63]:
print(o['layer4'].size())

torch.Size([1, 512, 30, 40])


In [66]:
import torchvision
torchvision.__version__


AttributeError: module 'torchvision' has no attribute '__version__'

In [31]:
{f'layer{k}': str(v) for v, k in enumerate([1, 2, 3, 4])}

{'layer1': '0', 'layer2': '1', 'layer3': '2', 'layer4': '3'}

In [ ]:
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [33]:
from PIL import Image
from torchvision import transforms
input_image = Image.open('detection/326.jpg')
# input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model



# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
# print(probabilities)

In [34]:
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
print(top5_catid)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

tensor([473, 600, 477, 726, 783], device='cuda:0')
can opener 0.41975638270378113
hook 0.07524648308753967
carpenter's kit 0.02796514891088009
plane 0.026483098044991493
screw 0.026465121656656265
